In [ ]:
import ipynb.fs.full.helper as h

from imp import reload
reload(h)

In [4]:
import os, cv2, shutil, random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np
from deepface import DeepFace
from retinaface import RetinaFace

In [ ]:
## get unknown pictures???

In [ ]:
original = '/tf/src/imagesUnknown/imagesUnknownOriginal/'
cropped = '/tf/src/imagesUnknown/imagesUnknownCropped2/'

myFiles = [f for f in os.listdir(original) if f.endswith('.jpg')]
    
for file in range(len(myFiles)):
        
    if not os.path.exists(cropped):
        os.mkdir(cropped)
    
    file_name = myFiles[file]
    try:
            #face = DeepFace.detectFace(img_path = imageFolder + file_name, target_size = (224, 224), detector_backend = "dlib")
            
            
        img = cv2.imread(original + file_name)
            
        faces = RetinaFace.detect_faces(img)
            
            #print(faces)
            
        for i in range(len(faces)):
            face = faces[f'face_{i+1}']['facial_area']
            (xmin,ymin,xmax,ymax) = face
            crop = img[ymin:ymax,xmin:xmax]
            crop = cv2.resize(crop,(224,224))
                
        plt.imsave(cropped + file_name, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
    except ValueError:
        print(ValueError)
        print(file_name)
            
    except TypeError:
        print(file_name)

In [ ]:
for folder in folderList:
    print(folder)
    pathFolder = pathImageFolder + '/' + folder 
    imageFolder = pathFolder + '/crop3/'
    #labelFolder = pathFolder + '/labels/' 
    myFiles = [f for f in os.listdir(imageFolder) if f.endswith('.jpg')]
    random.shuffle(myFiles)
    
    for i, file in enumerate(myFiles):
        if(i <= int(len(myFiles) * 0.7)):
            if not os.path.exists(f'{dataset}/train/{folder}'):
                os.mkdir(f'{dataset}/train/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/train/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/train')
        elif(i > int(len(myFiles) * 0.7) and i <= int(len(myFiles) * 0.85)):
            if not os.path.exists(f'{dataset}/val/{folder}'):
                os.mkdir(f'{dataset}/val/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/val/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/val')
        else:
            if not os.path.exists(f'{dataset}/test/{folder}'):
                os.mkdir(f'{dataset}/test/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/test/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/test')

In [ ]:
    imageFolder = '/tf/src/imagesUnknown/imagesUnknownCropped2'
    folder = 'Unknown'

    myFiles = [f for f in os.listdir(imageFolder) if f.endswith('.jpg')]
    random.shuffle(myFiles)
    
    for i, file in enumerate(myFiles):
        if(i <= int(len(myFiles) * 0.7)):
            if not os.path.exists(f'{dataset}/train/{folder}'):
                os.mkdir(f'{dataset}/train/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/train/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/train')
        elif(i > int(len(myFiles) * 0.7) and i <= int(len(myFiles) * 0.85)):
            if not os.path.exists(f'{dataset}/val/{folder}'):
                os.mkdir(f'{dataset}/val/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/val/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/val')
        else:
            if not os.path.exists(f'{dataset}/test/{folder}'):
                os.mkdir(f'{dataset}/test/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{dataset}/test/{folder}/{file}')
            #shutil.copy(f'{labelFolder}/{file.replace("jpg", "xml")}', f'{dataset}/test')


In [ ]:
##############################################

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
rows=224  #Number of Pixels in the Rows for Input. 
cols=224  #Number of Pixels in Columns for Output

In [ ]:
trained_model=MobileNetV2(input_shape=(rows,cols,3),           
                    include_top=False,
                    weights='imagenet')   


trained_model.trainable=True  #Un-Freeze all the pretrained layers of 'MobileNetV2 for Training.
    
trained_model.summary()

In [ ]:
last_layer=trained_model.get_layer('out_relu')   
last_layer_output=last_layer.output 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,

                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                rotation_range=20,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
                                 
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(f'{dataset}/train',
                                                  target_size=(rows,cols),
                                                  class_mode='categorical')

test_generator=test_datagen.flow_from_directory(f'{dataset}/test',
                                                target_size=(rows,cols),
                                                class_mode='categorical')

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)
#Add a Dropout layer.
x = tf.keras.layers.Dropout(0.2)(x)                      
# Add a final softmax layer for classification.
x = tf.keras.layers.Dense(73,activation='softmax')(x)


# todo inject labels?
# https://stackoverflow.com/questions/44310448/attaching-class-labels-to-a-keras-model

model = tf.keras.Model(trained_model.input,x) 

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
#Summary of the model.
model.summary()

In [ ]:
result=model.fit(train_generator,
                 validation_data=test_generator,
                 epochs=10,
                 verbose=1)

In [ ]:
import matplotlib.pyplot as plt

acc = result.history['accuracy']
val_acc = result.history['val_accuracy']

loss = result.history['loss']
val_loss = result.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(9, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save(f'{abspath}/models/classification/newClass/v9/v6/')

In [ ]:
model.evaluate(test_generator)